In [ ]:
# !pip install pyspark

In [ ]:
spark

In [ ]:
#importando bibliotecas
import os
import sys
import pytz
import numpy as np
import datetime
import time
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, udf, lpad, translate, to_timestamp, date_format
from datetime import datetime
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg, col, asc, desc, min, avg,max,sum,count,stddev

In [ ]:
#funcao freq
### FREQ FUNCTION
def Freq(pTabela,pColuna):

    qtd_total=pTabela.count()

    pTabela.registerTempTable("tab_input")
    frq = spark.sql(
            """
                select
                    {col},
                    count(*) as qtd_absoluto,
                    round(100*(count(*) / {tot}),2) as qtd_percentual
                from
                    tab_input
                group by
                    {col}
                order by
                    {col}
            """.format(col=pColuna, tot=qtd_total))

    qtd=frq.count()
    print('Quantidade de dominios',qtd)
    if qtd > 500:
        frq.show(1000,truncate=False)
        return "Dominio muito granular"

    else:
        frq.show(qtd,truncate=False)
        print("volumetria total:",qtd_total)
        return 'Freq da coluna ' + pColuna;

In [ ]:
agora = datetime.now(pytz.timezone('America/Sao_Paulo'))
dthproc = agora.strftime("%Y%m%d%H%M%S")

In [ ]:
spark = SparkSession.builder.appName("t_ssp_montly_file").getOrCreate()
sqlContext = SQLContext(spark.sparkContext)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
spark

#Carregando arquivos

In [ ]:
### MOD_PER_FILE
file = "dados_ssp_sp.csv"

In [ ]:
ts_file_generation = file.split("-")[-1].replace(".csv", "") + "00"
print(ts_file_generation)

dados_ssp_sp00


# Endereco Buckets (provisorio)

In [ ]:
bucket_control =    "bkt-ssp/Files/05_Control"
bucket_ingestion =  "bkt-ssp/Files/01_IngestionZone"
bucket_raw =        "bkt-ssp/Files/02_RawZone"
bucket_trusted =    "bkt-ssp/Files/03_TrustedZone"
bucket_refined =    "bkt-ssp/Files/04_RefinedZone"

In [ ]:
### MOD_PER_FILE
output_lake = "ingestion_ssp_montly_file"

In [ ]:
### MOD_PER_FILE
full_path_ingestion = 's3://{bkt}/{file}'.format(bkt=bucket_ingestion,file=file)
print(full_path_ingestion)

s3://bkt-ssp/Files/1_IngestionZone/dados_ssp_sp.csv


In [ ]:
start_time = time.time()

pouso = spark.read.format(
       "com.databricks.spark.csv").option(
       "header", "true").option(
       "encoding", "ISO-8859-1").option(
       "encoding", "UTF-8").option(
       "inferSchema", "false").option(
        "delimiter", ',').load(
       full_path_ingestion)

pouso.registerTempTable("pouso")
pouso.cache()
qtd = pouso.count()
print('records ingested: ', qtd)

# Calculate the elapsed time
elapsed_time = time.time() - start_time
print('Elapsed time:', elapsed_time, 'seconds')

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


records ingested:  153876
Elapsed time: 0.9863908290863037 seconds


# Ordernamento dados ascendente

In [ ]:
pouso = pouso.orderBy(asc("ano"),asc("mes"), asc("regiao_ssp"), asc("id_municipio"))

# Listagem Colunas

In [ ]:
for column in pouso.columns:
  print(column+',')

ano,
mes,
id_municipio,
regiao_ssp,
homicidio_doloso,
numero_de_vitimas_em_homicidio_doloso,
homicidio_doloso_por_acidente_de_transito,
numero_de_vitimas_em_homicidio_doloso_por_acidente_de_transito,
homicidio_culposo_por_acidente_de_transito,
homicidio_culposo_outros,
tentativa_de_homicidio,
lesao_corporal_seguida_de_morte,
lesao_corporal_dolosa,
lesao_corporal_culposa_por_acidente_de_transito,
lesao_corporal_culposa_outras,
latrocinio,
numero_de_vitimas_em_latrocinio,
total_de_estupro,
estupro,
estupro_de_vulneravel,
total_de_roubo_outros,
roubo_outros,
roubo_de_veiculo,
roubo_a_banco,
roubo_de_carga,
furto_outros,
furto_de_veiculo,


In [ ]:
### Lake SQL statement, partition files will be saved in raw bucket

lake = spark.sql(
    """
        select
            -- padrao para todas as tabelas

            --Nova Coluna para facilitar particionamento yyyy-mm
            concat(ano, lpad(mes, 2, '0')) as yyyymm,
            concat(ano, lpad(mes, 2, '0')) as yyyymm_partition,

            -- Data e Hora do Processamento

            {pdthproc} as ts_proc,
            {pdthproc} as ts_proc_partition,

            -- campos do arquivos

            ano,
            mes,
            id_municipio,
            regiao_ssp,
            homicidio_doloso,
            numero_de_vitimas_em_homicidio_doloso,
            homicidio_doloso_por_acidente_de_transito,
            numero_de_vitimas_em_homicidio_doloso_por_acidente_de_transito,
            homicidio_culposo_por_acidente_de_transito,
            homicidio_culposo_outros,
            tentativa_de_homicidio,
            lesao_corporal_seguida_de_morte,
            lesao_corporal_dolosa,
            lesao_corporal_culposa_por_acidente_de_transito,
            lesao_corporal_culposa_outras,
            latrocinio,
            numero_de_vitimas_em_latrocinio,
            total_de_estupro,
            estupro,
            estupro_de_vulneravel,
            total_de_roubo_outros,
            roubo_outros,
            roubo_de_veiculo,
            roubo_a_banco,
            roubo_de_carga,
            furto_outros,
            furto_de_veiculo
        from
            pouso
    """.format(tsfileger=ts_file_generation, pdthproc=dthproc))
lake.registerTempTable("lake")
lake.cache()
lake.count()

153876

In [ ]:
#Destino
path_raw = os.path.join('s3://',bucket_raw,output_lake)
print(path_raw)

s3://bkt-ssp/Files/2_RawZone/ingestion_ssp_montly_file


In [ ]:
path_raw = os.path.join('s3://',bucket_raw,output_lake)
print(path_raw)

lake.coalesce(1).write \
.partitionBy("yyyymm", "yyyymm_partition", "ts_proc", "ts_proc_partition") \
.csv(path_raw,mode='overwrite')

In [ ]:
#End

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>